# Описание проекта

###  Общая информация:
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

###  Цель исследования: 
Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

###  Данные:
Данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). 

###  Описание данных:
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

- `сalls` — количество звонков;
- `minutes` — суммарная длительность звонков в минутах; 
- `messages` — количество sms-сообщений;
- `mb_used` — израсходованный интернет-трафик в Мб;
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0); 

###  Ход исследования:
1. Открыть файл с данными и изучить общую информацию.
2. Предобработка данных не понадобится — мы её уже сделали.
3. Разделить исходные данные на обучающую, валидационную и тестовую выборки.
4. Исследовать качество разных моделей, меняя гиперпараметры. Кратко написать выводы исследования.
5. Проверить качество модели на тестовой выборке.
6. Дополнительное задание: проверить модели на вменяемость. 
7. Общий вывод.


# 1. Откройте файл с данными и изучите общую информацию.

**1.Загружаем данные из файла в датафрейм.**

In [1]:
# импортируем библиотеки в самом начале

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from tqdm import tqdm
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
# безопасно открываем файл
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

**2. Изучаем общую информацию о полученном датафрейме.**

2.1. Выводим первые 10 строчек датафрейма df на экран.

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


2.2. Выведем основную информацию о датафрейме.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


2.3. Узнаем, есть ли в данных пропуски и явные дубликаты.

In [5]:
# суммарные пропуски
df.isna().sum().sort_values(ascending=False)

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [6]:
# подсчет дубликатов
df.duplicated().sum() 

0

#  Вывод: 

Данных для исследования достаточно. В предобработке они не нуждаются. следовательно, мы можем продолжить дальнейшую работу.

# 2. Предобработка данных.

В ней нет необходимости, пропустим этот пункт.

# 3. Разделим исходные данные на обучающую, валидационную и тестовую выборки.

Данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1.


#### 3.1. Сначала отделим тестовую выборку. До этапа тестирования она нам не пригодится.

In [7]:
# тестовая выборка
train_valid, test = train_test_split(df, test_size = 0.20, random_state = 12345)


#### 3.2. Оставшиеся данные делим на обучающую и валидационную выборку.

In [8]:
# обучающая и валидационная выборка
train, valid = train_test_split(train_valid, test_size = 0.25, random_state = 12345)

#### 3.3. Сохраним признаки в отдельных переменных: целевой признак для нас - это "is_ultra", все остальные колонки, кроме целевой - нецелевые признаки.

In [9]:
# признаки для тестовой выборки
features_test = test.drop('is_ultra', axis=1)
target_test = test['is_ultra']

# признаки для обучающей выборки
features_train = train.drop('is_ultra', axis=1)
target_train = train['is_ultra']

# признаки для валидационной выборки
features_valid = valid.drop('is_ultra', axis=1)
target_valid = valid['is_ultra']

#### 3.4. Проверим коректность разбива данных.

In [10]:
# процент тестовой выборки
f'{features_test.shape[0]/df.shape[0]:.1%}'

'20.0%'

In [11]:
# процент обучающей выборки
f'{features_train.shape[0]/df.shape[0]:.1%}'

'60.0%'

In [12]:
# процент валидационной выборки
f'{features_valid.shape[0]/df.shape[0]:.1%}'

'20.0%'

#  Вывод: 

* Нам удалось разделить исходные данные на обучающую, валидационную и тестовую выборки в соотношение 3:1:1. 
* Также определить целевой и нецелевые признаки и подготовить данные для обучения, валидации и теста модели.

Можем продолжать работу.

# 4. Исследуем качество разных моделей, меняя гиперпараметры. Кратко напишем выводы исследования.

### 4.1. Рассмотрим одну популярную модель — решающее дерево. 

**Построим модели с различными значениями глубины дерева на валидационной выборке и найдем лучшее из всех.**

In [13]:
best_result = 0
best_depth = 1

# построим модели с глубиной дерева от 1 до 10
for depth in range(1, 10):
#  инициализируем модель DecisionTreeClassifier с параметрами random_state=12345 и max_depth=depth
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
# обучим модель на тренировочной выборке
    model_tree.fit(features_train, target_train) 
# получим предсказания модели на валидационной выборке
    predictions_valid = model_tree.predict(features_valid) 
# определим качество модели на валидационной выборке    
    accuracy_valid = model_tree.score(features_valid, target_valid) 
    
    if accuracy_valid > best_result:
# сохраним наилучшее значение метрики accuracy на валидационных     
        best_result = accuracy_valid
# сохраним наилучшую глубину дерева    
        best_depth = depth

print('Наилучшим значением accuracy для модели "дерево решений" является', best_result, 'с глубиной дерева:', best_depth)

Наилучшим значением accuracy для модели "дерево решений" является 0.7744945567651633 с глубиной дерева: 7


Это значение превышает значение 0.75 (о котором говорится в задании), что в целом нам подходит, но посмотрим и другие модели, возможно, результаты будут лучше.

### 4.2. Рассмотрим следующую модель — случайный лес. 

**Построим модели с различными значениями глубины дерева и количества деревьев на валидационной выборке и найдем лучшее из всех.**

In [14]:
best_est = 0
best_depth = 0
best_result = 0

for est in range(1, 11):
    for depth in range (1, 10):
# инициализируем модель RandomForestClassifier с параметрами random_state=12345, n_estimators=est и max_depth=depth
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
# обучим модель на тренировочной выборкее
        model_forest.fit(features_train, target_train) 
# получим предсказания модели на валидационной выборке
        predictions_valid = model_forest.predict(features_valid) 
# определим качество модели на валидационной выборке
        accuracy_valid = model_forest.score(features_valid, target_valid) 

        if accuracy_valid > best_result:
# сохраним наилучшее значение метрики accuracy на валидационных         
            best_result = accuracy_valid 
# сохраним наилучшее кол-во деревьев 
            best_est = est        
# сохраним наилучшую глубину дерева 
            best_depth = depth 

print('Наилучшим значением accuracy для модели "случайный лес" является', best_result, 'с глубиной дерева:', best_depth,
       'и с количеством деревьев:', best_est)

Наилучшим значением accuracy для модели "случайный лес" является 0.7962674961119751 с глубиной дерева: 9 и с количеством деревьев: 3


Это значение превышает значение 0.75 (о котором говорится в задании) и уже лучше значения модели "решающее дерево", что в целом нам подходит, но посмотрим и другие модели, возможно, результаты будут лучше.

### 4.3. Рассмотрим следующую модель —  логистическая регрессия. 

In [15]:
# первый гиперпараметр позволяет выбрать алгоритм, который будет строить модель. 
# алгоритм 'lbfgs' — один из самых распространённых. Он подходит для большинства задач. 
# гиперпараметром max_iter задаётся максимальное количество итераций обучения. 
# значение этого параметра по умолчанию равно 1000, но в некоторых случаях понадобится больше итераций
model_logistic = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
# обучим модель на тренировочной выборкее
model_logistic.fit(features_train, target_train) 
# получим предсказания модели на валидационной выборке
predictions_valid = model_logistic.predict(features_valid) 
# определим качество модели на валидационной выборке
accuracy_valid = model_logistic.score(features_valid, target_valid)

print('Наилучшим значением accuracy для модели "логистическая регрессия" является', accuracy_valid)

Наилучшим значением accuracy для модели "логистическая регрессия" является 0.7262830482115086


Это наименьшее полученное значение оценки качества из всех моделей. 

#  Вывод: 

Нам удалось исследовать качество разных моделей, меняя гиперпараметры:
- количества деревьев(n_estimators),
- максимальной глубины деревьев (max_depth),
- выбора алгоритма (solver),
- максимального количества итераций обучения (max_iter).

Удалось выяснить, что наибольшее значение оценки качества из всех моделей у модели "случайный лес" (0.7962674961119751), далее идет "дерево решений" (0.7744945567651633), следующая модель - "логистическая регрессия" (0.7262830482115086).

Следовательно, для тестовой выборки возьмем модель "случайный лес".

# 5. Проверим качество модели на тестовой выборке.

In [16]:
# получим предсказания модели на тестовой выборке
predictions_test = model_forest.predict(features_test) 
# определим качество модели на тестовой выборке
accuracy_test = model_forest.score(features_test, target_test) 

print('Наилучшим значением accuracy для модели "случайный лес" является', accuracy_test)

Наилучшим значением accuracy для модели "случайный лес" является 0.7947122861586314


#  Вывод: 

Для тестовой выборки значение оценки качества у модели "случайный лес" (0.7947122861586314), этот показатель выше 0.75 (который указан в задании). Значит, условие мы выполнили.

#  6. Проверим модели на адекватность.

**Для проверки используем модель DummyClassifier для задач классификации.**

In [17]:
# создадим модель model_dummy
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=12345)
# обучим модель на обучающей выборке
dummy_clf.fit(features_train, target_train)
# получим предсказания модели на тестовой выборке
dummy_clf.predict(features_test)
# определим качество модели на тестовой выборке
accuracy_dummy = dummy_clf.score(features_test, target_test)

print('Значением accuracy для модели "DummyClassifier" является', accuracy_dummy)

Значением accuracy для модели "DummyClassifier" является 0.6951788491446346


In [18]:
# код ревьюера
target_test.value_counts(normalize=True)[0]

0.6951788491446346

#  Вывод: 

Значение аccuracy модели DummyClassifier ниже, чем у модели model_forest, следовательно, модель адекватна и эффективна.

#  7. Общий вывод.

В ходе исследования с целью разработки модели, позволяющей наиболее точно предложить пользователям мобильной связи тарифы «Смарт» или «Ультра» были выполнены следующие шаги:

1. Изучена общая информация по датафрейму и отмечен ход работы.

2. Проведена предобработка данных - предоставленные данные чистые.

3. Были разделены исходные данные на обучающую, валидационную и тестовую выборки в соотношение 3:1:1.

4. Также определены целевой и нецелевые признаки.

5. Были исследованы модели Decision Tree Classifier ('дерево решений'), RandomForestClassifier ('случайный лес')и Logistic Regression ('логистическая регрессия').

6. По результатам исследования наилучшую эффективность (точность/скорость) показала модель RandomForestClassifier с гиперпараметрами n_estimators равным - 3 и max_depth равным 9. Значение accuracy составило 0.7962674961119751.

7. Лучшая модель RandomForestClassifier была проверена на тестовой выборке. Значение Accuracy составило 0.7947122861586314, этот показатель выше 0.75 (который указан в задании). Значит, условие мы выполнили..  

9. Была проведена проверка на вменяемость - для проверки использовалась модель DummyClassifier, которая показала результат Accuracy - 0.6951788491446346. Это значение хуже, чем результат нашей лучшей модели, следовательно, модель адекватна и эффективна..

Данные исследования подсвятили для нас лучшую модель, которая более эффективно подберет клиентам новый тариф.